In [123]:
import pandas as pd
import numpy as np
import math
import os

from PyRadioLoc.Utils.GeoUtils import GeoUtils

# Gera o mapa de fingerprints de 5x5, 10x10 e 20x20

In [118]:
DATA_DIR = os.getcwd() + "/database"

data = pd.read_csv(DATA_DIR + "/LocTreino_Equipe_4.csv")

LAT_INI  = min(data['lat'])
LAT_FIN  = max(data['lat'])
LONG_INI = min(data['lon'])
LONG_FIN = max(data['lon'])
delta_x = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_INI, LONG_FIN),
              GeoUtils.distanceInKm(LAT_FIN, LONG_INI, LAT_FIN, LONG_FIN))
delta_y = max(GeoUtils.distanceInKm(LAT_INI, LONG_INI, LAT_FIN, LONG_INI),
              GeoUtils.distanceInKm(LAT_INI, LONG_FIN, LAT_FIN, LONG_FIN))
R = [5, 10, 20]               # dimensoes de cada célula

In [28]:
# Para cada um das dimensoes de R calcula uma matriz de fingerprint e salva e um csv 
for num in R:    
    n_cells_x = math.ceil((delta_x * 1000) / num)
    n_cells_y = math.ceil((delta_y * 1000) / num)

    inc_lat = (LAT_FIN - LAT_INI) / n_cells_x
    inc_lng = (LONG_FIN - LONG_INI) / n_cells_y
    
    result = []

    for i in range(n_cells_x):
        for j in range(n_cells_y):
            lat = LAT_INI + i * inc_lat
            lng = LONG_INI + j * inc_lng
            result.append((lat, lng))

    # salva para o CSV
    fingerprint_map = pd.DataFrame(result, columns=['lat', 'lon'])
    fingerprint_map.to_csv(DATA_DIR + "/fingerprint_{}.csv".format(num), index=False)
    
    print(n_cells_x, n_cells_y)

325 291
163 146
82 73
